In [1]:
##########!!!!!!!!!!!!!!!! need to add to git https://www.theserverside.com/blog/Coffee-Talk-Java-News-Stories-and-Opinions/How-to-push-an-existing-project-to-GitHub
##i think this is done!

import pandas as pd
import numpy as np
#to get rid of chained assignment errors
pd.options.mode.chained_assignment = None  # default='warn'
df = pd.read_csv('data/survey_results_public.csv')
schema_df = pd.read_csv('data/survey_results_schema.csv', index_col='qname')
df.set_index('ResponseId', inplace=True)
pd.set_option('display.max_columns',85)# applys to both df's schema and results
pd.set_option('display.max_rows',85) # applys to both df's schema and results

#looking at general Df, getting the count of total responses to put counts into perspective
# df.columns
# schema_df.columns
# display(df.shape)
# display(schema_df.shape)
# df.head(10)
# schema_df(10)

#looking at country list, and how many have non np.NAN answers
# print(df['Country'].unique())
# display(df['Country'].count())
#note there are 1497 NaN's


#looking at countries with the most responses, taking top 45 for now, getting total sum from those 45
# display(df['Country'].value_counts(dropna=False,sort=True,ascending=False).head(45))
# display(np.sum(df['Country'].value_counts(dropna=True,sort=True,ascending=False).head(45)))

#Creating a df_45 DataFrame of the countries and thier respective number of partisipants.(Where I will add aggragated data). 
a = df['Country'].value_counts(dropna=True)
# display(a)
# print(type(a)) 
df_45 = a.to_frame().rename(columns={'Country': 'Count'}) #this turns series into a Df
#print(type(df_45))
# removing unwated rows
filt = df_45['Count']>316
test= df_45[filt]
#display(test)
#setting up df_45
df_45 = test
df_45.shape
df_45 = df_45.reset_index()
df_45 = df_45.rename(columns={'index': 'Country'})
#df_45

#creatig a list of the countries 
countries = test.index.tolist()
#countries

# updating original DF to only include rows from the 45 chosen countries
filt2 = df['Country'].isin(countries)
df_whole_45 = df[filt2]
#df_whole_45

# 64084 matches sum from above, should be correct
#df_whole_45.shape

In [2]:
# get % fully remote...add them to df_45
    # group by country
country_grp = df_whole_45.groupby(['Country'])

country_grp['RemoteWork'].value_counts(normalize = True) # puts it into a percentage
remote_work_perc = country_grp['RemoteWork'].value_counts(normalize=True)

# filter the DataFrame to only include rows where the value of the 'RemoteWork' column is 'Fully remote'
fully_remote_perc = remote_work_perc.loc[:, 'Fully remote'].rename('fully_remote_perc')

print(type(fully_remote_perc))
df_45 = df_45.join(fully_remote_perc, on='Country')
Work_Location = country_grp['RemoteWork'].count().rename('Work_Location Count_nonNA')

df_45 = df_45.join(Work_Location, on='Country')
# fully_remote_perc series added to df_45


<class 'pandas.core.series.Series'>


In [3]:
pd.set_option('display.float_format', lambda x: f'{x:.2f}')
# looking at currencies in the new df to see how many I need to look up
#print(df_whole_45['Currency'].unique())
#print(df_whole_45['Currency'].value_counts())
 # NOTE: not going to use any currencies with less than 6 responses. sorry it is just for the time.

# define dict of currency converters
currencies_converters = {'EUR': .93, 'USD': 1, 'INR': 82.31, 'GBP': .82,'CAD': 1.37, 'BRL': 5.25, 'PLN': 4.39, 'AUD': 1.5,
                         'SEK': 10.4, 'RUB': 80.2, 'CHF': .91, 'TRY': 19.24,'ILS' : 3.58, 'CZK': 21.47, 'IRR': 42250,
                         'PKR': 288.25, 'DKK': 6.83,'MXN': 18.31, 'NOK': 10.44,'ZAR': 18.05, 'NZD':1.58, 'RON': 4.53, 'BDT': 107.52, 'HUF': 344.99 ,
                         'CNY': 6.88, 'IDR': 14959, 'UAH': 36.95 , 'JPY': 131.30, 'ARS': 211.23, 'BGN': 1.79, 'COP': 4584.75, 'PHP': 54.50,
                         'EGP': 30.90, 'NGN': 464.50, 'VND': 23467.50, 'AED': 3.67} # taken from goodle converter 3/14/2023 and 4/5

#triming unneeded characters from 'Currency'
df_whole_45['Currency'] = df_whole_45['Currency'].str[:3]

# function to convert currency
def convert_currency(currency):
    if currency in currencies_converters:
        return currencies_converters[currency]
    else:
        return np.NaN  # using np.NaN instead of NaN
    
# add new 'CurrencyConvert' column to DataFrame using apply() method
df_whole_45['CurrencyConvert'] = df_whole_45['Currency'].apply(convert_currency)


def get_interval(interval):
    if pd.isna(interval):
        return np.NaN
    elif interval == "Yearly":
        return 1
    elif interval == "Monthly":
        return 12
    elif interval == "Weekly":
        return 52
    else:
        raise ValueError("Invalid interval value: {}".format(interval))
        
        
        
# use function - testing if the comp freq gets more results when removed
df_whole_45['YearlyTotConvertCOMP'] = (1/df_whole_45['CurrencyConvert']) * df_whole_45['CompTotal'] * df_whole_45['CompFreq'].apply(get_interval)
# use function - testing if the comp freq gets more results when removed
#df_whole_45['YearlyTotConvertCOMP'] = (1/df_whole_45['CurrencyConvert']) * df_whole_45['CompTotal'] 
#trying to get rid of scientific notation:
df_whole_45['YearlyTotConvertCOMP'] = df_whole_45['YearlyTotConvertCOMP'].apply(lambda x: float(x))


#looking at us, italy, Iran,Nigeria...there is something very wrong with my avearges!!!
#us_data = country_grp.get_group('United States of America')['YearlyTotConvertCOMP'].sort_values(ascending=False)
# Print the YearlyTotConvertCOMP values for USA
#print(us_data.head(55))
#NOTE: looks like there are some intentional erroneous answers in the extreem,
#      as well as possibly some that indicated the incorrect currency or interval. I am removing anything over $999,999.

# removing 'YearlyTotConvertCOMP' under$1500 and over $999,999
print('shape before outliers removal')
display(df_whole_45.shape)
filt3 = (df_whole_45['YearlyTotConvertCOMP']<999999) & (df_whole_45['YearlyTotConvertCOMP']>1500)
df_whole_45 = df_whole_45[filt3]
print('shape after outliers removal')
display(df_whole_45.shape)

# display updated DataFrame slice
df_whole_45[['Country', 'Currency','CurrencyConvert', 'CompTotal', 'CompFreq','YearlyTotConvertCOMP']].sort_values(by = 'YearlyTotConvertCOMP' , ascending=True).head(75)
#df = df.dropna(axis=1).....This will remove all columns that contain at least one NaN value.

print(df_whole_45['YearlyTotConvertCOMP'].max())

print(df_whole_45['YearlyTotConvertCOMP'].min()) 

#looking at mean and median
country_grp = df_whole_45.groupby(['Country'])
country_grp['YearlyTotConvertCOMP'].mean().round(0)
#pd.set_option('display.float_format', lambda x: f'{x:.2f}')
country_grp['YearlyTotConvertCOMP'].agg(['mean', 'median']).round(0)

#Salary series added to df_45
Mean_Salary = country_grp['YearlyTotConvertCOMP'].mean().round(1).rename('Mean Salary')
Median_Salary = country_grp['YearlyTotConvertCOMP'].median().round(1).rename('Median Salary')
Salary_Count_NonNA = country_grp['Age'].count().rename('Salary_Count NonNA')
df_45 = df_45.join(Mean_Salary, on='Country')
df_45 = df_45.join(Median_Salary, on='Country')
df_45 = df_45.join(Salary_Count_NonNA, on='Country')


shape before outliers removal


(64084, 80)

shape after outliers removal


(33561, 80)

996000.0
1553.3980582524273


In [4]:
#looking at YearsCode:
# df_whole_45['YearsCode'].unique()

# fix data in 'years code'
df_whole_45.loc[df_whole_45['YearsCode'] == 'Less than 1 year', 'YearsCode'] = 0
df_whole_45.is_copy = False
df_whole_45.loc[df_whole_45['YearsCode'] == 'More than 50 years', 'YearsCode'] = 51
df_whole_45.is_copy = False
df_whole_45.loc[:, 'YearsCode'] = df_whole_45['YearsCode'].astype(float)
#df_whole_45['YearsCode'] = df_whole_45['YearsCode'].astype(float)
df_whole_45.is_copy = False

#df_whole_45.loc[:, 'Age'] = df_whole_45['Age']
    
#checking again
#df_whole_45['YearsCode'].unique()
#df_whole_45.dropna(subset=['Age'], inplace=True)

#get Mean and Median
#display(df_whole_45['YearsCode'].mean())
#df_whole_45['YearsCode'].median()  

#Looking at results:
country_grp = df_whole_45.groupby(['Country'])
#country_grp['YearsCode'].mean().round(1)
#country_grp['YearsCode'].median().round(1)
country_grp['YearsCode'].agg(['mean', 'median']).round(1)
# checking participation 
country_grp['YearsCode'].count()


#yearsCode series added to df_45
Mean_YearsCode = country_grp['YearsCode'].mean().round(1).rename('Mean YearsCode')
Median_YearsCode = country_grp['YearsCode'].median().round(1).rename('Median YearsCode')
YearsCode_Count_NonNA = country_grp['YearsCode'].count().rename('YearsCode_Count NonNA')
df_45 = df_45.join(Mean_YearsCode, on='Country')
df_45 = df_45.join(Median_YearsCode, on='Country')
df_45 = df_45.join(YearsCode_Count_NonNA, on='Country')


C:\Users\Adison\AppData\Local\Temp\ipykernel_8940\2337236460.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_whole_45.loc[:, 'YearsCode'] = df_whole_45['YearsCode'].astype(float)


In [5]:
#looking at age distribution
# What is the data type? It is in ranges.
df_whole_45['Age'].unique()
df_whole_45['Age'].value_counts()

#looking at country group 'age'
#country_grp['Age'].value_counts().head(50)
# Note: after looking at the responses split by coutry, they all have the same most common answer of '25-34 years old'.
        # This is not helpful or intersting. I am going to replace each range with the year in the center of the range 
        # and then take the average. This will represent the spread and weight of the age in each country.
        
def change_age(age):
    if pd.isna(age):
        return np.NaN
    elif age == "Prefer not to say":
        return np.NaN
    elif age == "Under 18 years old":
        return 16
    elif age == "18-24 years old":
        return 21
    elif age == "25-34 years old":
        return 29.5
    elif age == "35-44 years old":
        return 39.5
    elif age == "45-54 years old":
        return 49.5
    elif age == "55-64 years old":
        return 55.9
    elif age == "65 years or older":
        return 70
    else:
        raise ValueError("Invalid interval value: {}".format(age))    
        
#overwriting ages in df_whole_45 from ranges in to floats. This is permanant and need only run once per eval.
 # If "text Change_age already complete" appears then it has run already.
    
if df_whole_45.iloc[1]['Age'] in [16,21,29.5,39.5,49.5,55.9,70]:
        print("Change_age already complete")
else:df_whole_45.loc[:, 'Age'] = df_whole_45['Age'].apply(change_age)


#looking at age distribution after change, looks good
# df_whole_45['Age'].unique()
# df_whole_45['Age'].value_counts()
# country_grp['Age'].value_counts().head(50)

#getting average by country:
#country_grp['Age'].mean().round(1)

# recording participation 
country_grp = df_whole_45.groupby(['Country'])
country_grp['Age'].count()

#age series added to df_45
Mean_Age = country_grp['Age'].mean().round(1).rename('Mean Age')
Age_Count_NonNA = country_grp['Age'].count().rename('Age_Count NonNA')
df_45 = df_45.join(Mean_Age, on='Country')
df_45 = df_45.join(Age_Count_NonNA, on='Country')


C:\Users\Adison\AppData\Local\Temp\ipykernel_8940\1208129647.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  else:df_whole_45.loc[:, 'Age'] = df_whole_45['Age'].apply(change_age)


In [6]:
# education level
country_grp = df_whole_45.groupby(['Country'])
#country_grp['EdLevel'].unique()
#country_grp['EdLevel'].value_counts().head(40)
#country_grp = df_whole_45.groupby(['Country'])

Top_Education_per_country = country_grp['EdLevel'].apply(lambda x: x.value_counts().index[0])

# Replace original education levels with simplified versions
replace_dict = {
    'Some college/university study without earning a degree': 'Some college',
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 'Bachelor’s',
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 'Master’s'
}
Top_Education_per_country = Top_Education_per_country.replace(replace_dict).rename('Most_Common ED_Level')
df_45 = df_45.join(Top_Education_per_country, on='Country')
Ed_level_Count_NonNA= country_grp['EdLevel'].count().rename('Ed_level Count NonNA')
df_45 = df_45.join(Ed_level_Count_NonNA, on='Country')
print(Top_Education_per_country)






Country
Argentina                                               Some college
Australia                                                 Bachelor’s
Austria                                                     Master’s
Bangladesh                                                Bachelor’s
Belgium                                                   Bachelor’s
Brazil                                                    Bachelor’s
Bulgaria                                                  Bachelor’s
Canada                                                    Bachelor’s
China                                                     Bachelor’s
Colombia                                                  Bachelor’s
Czech Republic                                              Master’s
Denmark                                                     Master’s
Egypt                                                     Bachelor’s
Finland                                                     Master’s
France                    

In [7]:
# get top listed language used, ,

In [26]:
# get top listed job (MainBranch):
#This is less intersting when devided by country, I want to make a pie chart of participation by listed responsibilites/job title.
df['DevType'].value_counts().head(40)
 
# make a copy of the DataFrame
df_copy = df.copy()

# drop NaN values
df_copy = df_copy.dropna(subset=['DevType'])

# Split the values in the 'CodingActivities' column by the delimiter ';'
df_copy['DevType'] = df_copy['DevType'].str.split(';')

# Use the explode() method to create a new row for each value in the resulting lists
df_copy = df_copy.explode('DevType')

# Use the value_counts() method to count the occurrence of each value
counts = df_copy['DevType'].value_counts()

# Divide each count by the total number of responses and multiply by 100 to get the percentage
percentages = (counts / len(df_copy)) * 100

print(percentages.head(45))

Developer, full-stack                           17.42
Developer, back-end                             16.14
Developer, front-end                             9.66
Developer, desktop or enterprise applications    5.79
Developer, mobile                                4.63
DevOps specialist                                3.74
Student                                          3.40
Cloud infrastructure engineer                    3.21
Database administrator                           2.99
System administrator                             2.98
Developer, embedded applications or devices      2.38
Project manager                                  2.36
Designer                                         2.28
Engineer, data                                   2.18
Engineering manager                              2.17
Data scientist or machine learning specialist    2.08
Data or business analyst                         1.94
Developer, QA or test                            1.88
Academic researcher         

In [8]:
#taking a look at df_45
df_45

,Country,Count,fully_remote_perc,Work_Location Count_nonNA,Mean Salary,Median Salary,Salary_Count NonNA,Mean YearsCode,Median YearsCode,YearsCode_Count NonNA,Mean Age,Age_Count NonNA,Most_Common ED_Level,Ed_level Count NonNA
0,United States of America,13543,0.58,11629,160561.30,140000.00,8204,16.90,14.00,8221,36.30,8179,Bachelor’s,8235
1,India,6639,0.40,4899,38270.20,18223.80,2112,8.80,8.00,2109,28.20,2109,Bachelor’s,2113
2,Germany,5395,0.34,4530,98732.30,69677.40,2870,15.60,14.00,2868,33.80,2868,Master’s,2870
3,United Kingdom of Great Britain and Northern I...,4190,0.43,3575,132061.60,79268.30,2563,16.40,14.00,2568,34.80,2557,Bachelor’s,2572
4,Canada,2490,0.62,2096,107117.40,86861.30,1419,16.00,14.00,1422,35.00,1417,Bachelor’s,1423
5,France,2328,0.23,1954,78165.20,51612.90,1358,13.80,12.00,1357,32.60,1357,Master’s,1357
6,Brazil,2109,0.67,1869,39287.40,25142.90,1274,12.20,10.00,1272,31.50,1274,Bachelor’s,1275
7,Poland,1732,0.55,1452,57862.60,46419.10,975,12.40,11.00,974,31.60,974,Master’s,975
8,Netherlands,1555,0.19,1334,93454.00,64828.00,887,15.30,13.00,886,33.80,885,Bachelor’s,886
9,Spain,1521,0.50,1353,94820.50,48387.10,983,13.90,12.00,984,34.20,981,Bachelor’s,983


In [ ]:
# make a copy of the DataFrame
df_copy = df.copy()

# Replace NaN values with an empty string
df_copy['CodingActivities'] = df_copy['CodingActivities'].fillna('')

# Split the values in the 'CodingActivities' column by the delimiter ';'
df_copy['CodingActivities'] = df_copy['CodingActivities'].str.split(';')

# Use the explode() method to create a new row for each value in the resulting lists
df_copy = df_copy.explode('CodingActivities')

# Use the value_counts() method to count the occurrence of each value
counts = df_copy['CodingActivities'].value_counts()

print(counts)